# DEPLOYMENT SYSTEM TIMESERIES



In [ ]:
📁 project-folder/
│
├── model/
│   ├── rf_model.pkl
├──__pycache__/ 
│   ├── model_utils.cpython-312.pyc
├── app.py
├── model_utils.py
├── NO2_Pademawu.csv
├── requirements.txt


##  File: `requirements.txt`
Ini penting karena Streamlit Cloud akan otomatis menginstal semua library dari file ini.

In [ ]:
streamlit
pandas
matplotlib
joblib
scikit-learn
statsmodels
numpy



##  File: `model_utils.py`

Di bawah ini adalah **isi lengkap** dari `model_utils.py`. Setiap bagian kode disertai penjelasan singkat setelah cuplikan kode di bagian komentar atau markdown berikutnya.


In [ ]:
# model_utils.py
import pandas as pd
import numpy as np
import joblib
import os
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.stattools import acf

# --- KONFIGURASI GLOBAL ---
N_LAGS = 7  # Jumlah hari sebelumnya (lags) yang digunakan sebagai fitur
TEST_SIZE_DAYS = 90 # Jumlah hari untuk data uji
MODEL_PATH = 'model/rf_model.pkl'
Z_SCORE = 1.96 # Faktor Z-score untuk 95% Confidence Interval

# ----------------------------------------------------
# A. FUNGSI UMUM DATA PRE-PROCESSING
# ----------------------------------------------------

def load_and_clean_data(file_path):
    """Memuat dan membersihkan data NO2."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File data tidak ditemukan: {file_path}")

    df = pd.read_csv(file_path)
    df['time'] = pd.to_datetime(df['time'])
    df = df.set_index('time')
    # Mengisi nilai hilang (missing values) dengan forward fill
    df['NO2'] = df['NO2'].fillna(method='ffill')
    return df

def create_lags(data, n_lags):
    """Membuat fitur lagged dari data deret waktu."""
    df_lags = pd.DataFrame(data['NO2'])
    for i in range(1, n_lags + 1):
        df_lags[f'NO2_Lag_{i}'] = df_lags['NO2'].shift(i)
    df_lags.dropna(inplace=True)
    return df_lags.drop('NO2', axis=1), df_lags['NO2']

def split_data(X, y, test_size_days):
    """Membagi data latih dan uji secara kronologis."""
    return X.iloc[:-test_size_days], X.iloc[-test_size_days:], \
           y.iloc[:-test_size_days], y.iloc[-test_size_days:]

def get_last_data(df, n_lags):
    """Mengambil N_LAGS data historis terakhir."""
    return df['NO2'].tail(n_lags)

# ----------------------------------------------------
# B. FUNGSI RANDOM FOREST (RF)
# ----------------------------------------------------

def train_rf(X_train, y_train):
    """Melatih Random Forest."""
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    joblib.dump(rf_model, MODEL_PATH)
    return rf_model

def predict_rf_n_days(model, last_data_series, n_days, n_lags):
    """Prediksi RF multi-step rekursif dengan Confidence Interval 95%."""
    if model is None or len(last_data_series) != n_lags:
        return pd.DataFrame()

    current_features = last_data_series.values
    predictions_data = []
    last_date = last_data_series.index[-1]

    for _ in range(n_days):
        X_input = current_features.reshape(1, -1)

        # 1. Prediksi dari setiap pohon (untuk CI)
        all_tree_preds = [tree.predict(X_input)[0] for tree in model.estimators_]

        # 2. Hitung Statistik
        mean_prediction = np.mean(all_tree_preds)
        std_prediction = np.std(all_tree_preds)

        # 3. Hitung Batas 95% CI
        lower_bound = mean_prediction - Z_SCORE * std_prediction
        upper_bound = mean_prediction + Z_SCORE * std_prediction

        predictions_data.append({
            'NO2_Prediction': mean_prediction,
            'Lower_Bound': lower_bound,
            'Upper_Bound': upper_bound
        })

        # 4. Update fitur rekursif (gunakan mean_prediction sebagai Lag_1 hari berikutnya)
        current_features = np.roll(current_features, 1)
        current_features[0] = mean_prediction

    forecast_dates = pd.date_range(start=last_date + timedelta(days=1), periods=n_days, freq='D')
    return pd.DataFrame(predictions_data, index=forecast_dates)

# ----------------------------------------------------
# C. FUNGSI UTAMA (TRAINING & LOADING)
# ----------------------------------------------------

def prepare_and_train_all(file_path, n_lags, test_size_days):
    """Fungsi utama untuk melatih RF dan evaluasi."""
    df = load_and_clean_data(file_path)

    # --- RF Training ---
    X_rf, y_rf = create_lags(df, n_lags)
    X_train_rf, X_test_rf, y_train_rf, y_test_rf = split_data(X_rf, y_rf, test_size_days)
    rf_model = train_rf(X_train_rf, y_train_rf)

    # Evaluasi metrik
    y_pred_rf = rf_model.predict(X_test_rf)
    mape_rf = mean_absolute_percentage_error(y_test_rf, y_pred_rf) * 100
    acf_rf = acf(y_test_rf - y_pred_rf, nlags=1, fft=True)[1]

    # Hasil
    last_data = get_last_data(df, n_lags)

    return {
        'rf_model': rf_model,
        'full_df': df,
        'last_data': last_data,
        'metrics_rf': {'mape': mape_rf, 'acf': acf_rf},
    }

def load_rf_model():
    """Memuat model RF yang sudah disimpan."""
    try:
        rf_model = joblib.load(MODEL_PATH)
        return rf_model
    except FileNotFoundError:
        return None

**FUNGSI PRE-PROCESSING DATA**
- Pembersihan dan pembentukan fitur data
- Memuat file CSV, ubah kolom time jadi datetime, set sebagai index, dan isi nilai kosong dengan forward fill
**Output**: DataFrame bersih dengan kolom NO2.
- Membuat fitur “lagged” — yaitu nilai NO2 dari hari-hari sebelumnya sebagai input model.
**Output** : X (fitur lag), y (target NO₂ saat ini)
- Pisahkan data train & test berdasarkan waktu (bukan acak).
**Output**: X_train, X_test, y_train, y_test
- Mengambil sejumlah n_lags data terakhir — digunakan untuk memulai prediksi ke depan.

**FUNGSI RANDOM FOREST**
- Latih model Random Forest menggunakan data training dan simpan model ke file .pkl.
**output** : Objek model terlatih (rf_model)
- Melakukan prediksi ke depan selama n_days hari secara rekursif.
Juga menghitung Confidence Interval 95% dari hasil setiap pohon di Random Forest.
**output** : DataFrame berisi prediksi + batas bawah/atas CI untuk tiap hari:
| Date | NO2_Prediction | Lower_Bound | Upper_Bound |

**FUNGSI UTAMA (TRAINING & LOADING)**
Menjalankan semua proses sekaligus:
- Load data
- Buat lag
- Split train-test
- Latih model
- Evaluasi (MAPE & ACF)
- Ambil data terakhir untuk prediksi berikutnya
- Memuat kembali model yang sudah disimpan di model/rf_model.pkl.


## File: `app.py` (Streamlit)

Berikut adalah isi lengkap `app.py`. File ini adalah antarmuka pengguna yang mengimpor model dari `model/` dan menampilkan hasil prediksi.


In [ ]:
# ==========================================================
# app.py 
# Sistem Prediksi NO2 berbasis Random Forest
# ==========================================================
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import os
import model_utils as mu
from datetime import date, timedelta
import joblib

# --- KONFIGURASI STREAMLIT ---
# Menggunakan layout "wide" untuk memaksimalkan ruang
st.set_page_config(
    page_title="Sistem Prediksi NO2",
    layout="wide"
)

# --- UTILITY: Streamlit Caching ---
@st.cache_resource
def load_or_train_model(file_path, n_lags, test_size):
    """Memuat atau melatih model Random Forest dan data pendukung."""
    if not os.path.exists('model'):
        os.makedirs('model')

    try:
        # Latih ulang untuk memastikan fitur terbaru
        results = mu.prepare_and_train_all(file_path, n_lags, test_size)
        return results['rf_model'], results['full_df'], results['last_data'], results['metrics_rf']

    except Exception as e:
        st.error(f"Sistem tidak dapat berjalan. Pastikan file '{file_path}' ada dan formatnya benar. Error: {e}")
        return None, None, None, None

# ==========================================================
# 0. UI UTAMA & INIT
# ==========================================================
st.title("🏭 Sistem Prediksi NO2 Harian")
st.caption("Aplikasi ini menggunakan model **Random Forest** untuk memprediksi konsentrasi NO₂ harian.")
st.markdown("---")

# Konfigurasi Awal
DATA_FILE = "NO2_Pademawu.csv"
N_LAGS = mu.N_LAGS
TEST_SIZE = mu.TEST_SIZE_DAYS

# --- Memuat/Melatih Model di Badan Utama ---
# Menggunakan spinner agar loading terlihat jelas
with st.spinner("Memuat dan melatih model..."):
    rf_model, full_data, last_data, metrics = load_or_train_model(DATA_FILE, N_LAGS, TEST_SIZE)

if rf_model is None:
    st.error("Model Gagal Dimuat/Dilatih. Aplikasi dihentikan.")
    st.stop()

st.success("Model Random Forest siap digunakan.")

# Ambil tanggal terakhir historis
last_historical_date = last_data.index[-1].date()


# ==========================================================
# 1. METRIK KINERJA MODEL
# ==========================================================
st.subheader("✅ Kinerja Model pada Data Uji")
st.markdown("Hasil evaluasi model Random Forest pada data uji terakhir:")
col1, col2 = st.columns(2)

with col1:
    st.metric(
        label="MAPE",
        value=f"{metrics['mape']:.2f} %",
        help="Mean Absolute Percentage Error. Persentase rata-rata kesalahan prediksi."
    )
with col2:
    st.metric(
        label="ACF Residuals (Lag 1)",
        value=f"{metrics['acf']:.4f}",
        help="Autokorelasi Residual pada lag 1. Nilai mendekati nol menunjukkan model baik menangkap pola."
    )

st.markdown("---")

# ==========================================================
# 2. INPUT PERIODE PREDIKSI
# ==========================================================
st.header("🎯 Tentukan Periode Prediksi")

input_container = st.container()

with input_container:
    col_start, col_end = st.columns([1, 2])

    # Tanggal Awal Otomatis
    start_date_forecast = last_historical_date + timedelta(days=1)

    with col_start:
        st.date_input(
            "Tanggal Mulai Prediksi (Otomatis)",
            value=start_date_forecast,
            disabled=True,
            key="start_date_fixed"
        )
        st.markdown(f"**Data Historis Terakhir:** `{last_historical_date}`") # Tampilkan info ini di sini

    # Tanggal Akhir
    with col_end:
        target_date = st.date_input(
            "Pilih Tanggal Akhir Prediksi",
            min_value=start_date_forecast,
            value=start_date_forecast + timedelta(days=7),
            max_value=start_date_forecast + timedelta(days=60),
            help="Pilih tanggal di masa depan, maksimal 60 hari dari data historis terakhir."
        )

days_to_forecast = (target_date - last_historical_date).days

# ==========================================================
# 3. TOMBOL & PROSES PREDIKSI
# ==========================================================
st.markdown("---")

if st.button(f"🚀 Mulai Prediksi NO₂ untuk {days_to_forecast} Hari (Hingga {target_date})", type="primary"):

    if days_to_forecast < 1:
        st.error("Jumlah hari prediksi tidak valid. Pilih tanggal akhir yang lebih jauh dari tanggal historis terakhir.")
        st.stop()

    with st.spinner(f"Memprediksi NO2 untuk **{days_to_forecast} hari** menggunakan Random Forest..."):

        # --- Prediksi Random Forest ---
        forecast_df = mu.predict_rf_n_days(
            rf_model, last_data, days_to_forecast, N_LAGS
        )

        # Pastikan index datetime dan urutan benar
        forecast_df.index = pd.to_datetime(forecast_df.index)
        forecast_df = forecast_df.sort_index()

    st.success("✅ Prediksi Selesai! Lihat hasilnya di bawah.")

    # Data historis terakhir (90 hari)
    historic_data_plot = full_data['NO2'].tail(90)

    # Format dataframe untuk ditampilkan
    display_df = forecast_df[['NO2_Prediction']].copy()
    display_df.index.name = 'Tanggal'
    display_df.columns = ['Prediksi NO2 (µg/m³)']

    # Tambahkan kolom level kualitas udara (Contoh Sederhana)
    def quality_level(no2):
        if no2 < 40: return "Baik"
        elif no2 < 80: return "Sedang (Moderate)"
        else: return "Tidak Sehat (Unhealthy)"

    display_df['Kualitas Udara'] = display_df['Prediksi NO2 (µg/m³)'].apply(quality_level)


    # ==========================================================
    # 4. HASIL PREDIKSI DENGAN TABS
    # ==========================================================
    st.header("📈 Hasil Prediksi")

    # Membuat Tabs
    tab_graph, tab_table, tab_download = st.tabs(["Grafik Prediksi", "Tabel Detail", "Unduh Hasil"])

    with tab_graph:
        st.subheader(f"Grafik Prediksi NO2 ({days_to_forecast} Hari)")
        fig, ax = plt.subplots(figsize=(10, 5))

        # Data Historis
        ax.plot(historic_data_plot.index, historic_data_plot.values,
                label='Historis (90 Hari Terakhir)', color='#1f77b4', linewidth=2, alpha=0.7)

        # Garis batas awal prediksi
        ax.axvline(x=forecast_df.index.min(), color='black', linestyle='--', linewidth=1, label='Awal Prediksi')

        # Plot hasil prediksi
        ax.plot(forecast_df.index, forecast_df['NO2_Prediction'],
                label='Prediksi NO2', color='red', linestyle='-', linewidth=2)

        ax.set_title(f"Prediksi NO2 Hingga {target_date}", fontsize=14)
        ax.set_xlabel("Tanggal", fontsize=12)
        ax.set_ylabel("Konsentrasi NO2 (µg/m³)", fontsize=12)
        ax.grid(True, linestyle=':', alpha=0.6)
        ax.legend(loc='upper left', fontsize='small')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        st.pyplot(fig)

    with tab_table:
        st.subheader("Detail Prediksi Harian")
        st.dataframe(display_df, use_container_width=True)

    with tab_download:
        st.subheader("Unduh Data")
        csv = display_df.to_csv().encode('utf-8')
        st.download_button(
            label="📥 Unduh Hasil Prediksi Lengkap (.csv)",
            data=csv,
            file_name=f'NO2_Prediksi_RF_Hingga_{target_date}.csv',
            mime='text/csv',
        )


**Fungsi Utama: `load_or_train_model()`:**

Melatih atau memuat model Random Forest dari file CSV.
- Mengambil data mentah (NO2_Pademawu.csv)
- Memanggil mu.prepare_and_train_all() (fungsi utama di model_utils.py)
- Menghasilkan:
    - rf_model → model Random Forest yang sudah dilatih
    - full_df → data historis NO₂ lengkap
    - last_data → 7 hari terakhir untuk prediksi selanjutnya

**Tampilkan Metrik Kinerja Model**

Menampilkan performa model pada data uji:
- MAPE → seberapa besar kesalahan rata-rata (%)
- ACF Residuals → autokorelasi residual, semakin mendekati 0 semakin baik






## Langkah Akhir: Menjalankan & Deploy

1. Pastikan environment berisi semua dependensi (lihat `requirements.txt` jika ada).  
2. Pastikan file dataset (`NO2_Pademawu.csv`) dan model (`model/*.pkl`) tersedia di folder proyek.  
3. Jalankan aplikasi lokal dengan:
```bash
streamlit run app.py
```



## Link Deploy
https://ameliasafitri-timeseries-23-039.streamlit.app/